In [1]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import *
from itertools import product

In [2]:
gamma_high = 0.1
gamma_low = 0.02

In [3]:
def create_operators(number_of_spins):
   
    operators = []
    for i in range(number_of_spins):
        # Start with identity for left side
        left_identity = [qeye(2)] * i

        # Start with identity for right side
        right_identity = [qeye(2)] * (number_of_spins - i - 1)

        # Create operators for x, y, z components
        spin_x = tensor(*(left_identity + [sigmax()] + right_identity))
        spin_y = tensor(*(left_identity + [sigmay()] + right_identity))
        spin_z = tensor(*(left_identity + [sigmaz()] + right_identity))

        # Append the operators for this spin as a triplet
        operators.append([spin_x, spin_y, spin_z])

    return operators

In [4]:
# print(create_operators(2))


In [5]:
def create_gamma(Like_list):
    gammas = []
    gammas.append(0.1)
    for element in Like_list:
        if element == "like":
            gammas.append(0.1)
        else:
            gammas.append(0.02)
    return gammas

In [6]:
# print(create_gamma(["unlike","like"]))

In [7]:
def create_theta(number_of_spins):
    length = int(((number_of_spins)*(number_of_spins - 1))/2)
    thetas = []
    for i in range(length):
        temp = []
        for j in range(length):
            if i==j:
                temp.append("NA")
            else:
                temp.append(np.pi/2)
        thetas.append(temp)
    return thetas

In [ ]:
# print(create_theta(3))

In [ ]:
# # Lengths between spins 01 02 12 Make it a matrix
# [1,100,100]
# [1,1,1]
# [100,100,100]
# [100,100,1]
# [100,1,100]

In [10]:
def get_coefficients(number_of_spins,gammas,thetas,distances):
    coefficients = [] # A's
    for i in range(number_of_spins):
        temp = []
        for j in range(number_of_spins):
            if i==j:
                temp.append("NA")
            else:
                temp.append(gammas[i]*gammas[j]*(1-3*np.cos(thetas[i][j])**2)/(distances[i][j]**3))
        coefficients.append(temp)
                
    return coefficients

In [11]:
def get_hamiltonians(number_of_spins,gammas,thetas,distances,operators):
    coefficients = get_coefficients(number_of_spins,gammas,thetas,distances)
    
    hamiltonians = []
    for i in range(number_of_spins):
        temp = []
        for j in range(number_of_spins):
            if j==i:
                temp.append("NA")
            else:
                if gammas[i] == gammas[j]: #Like
                    temp.append(coefficients[i][j]*( 2*operators[i][2]*operators[j][2] - operators[i][0]*operators[j][0] - operators[i][1]*operators[j][1])/4)
                else:
                    temp.append(coefficients[i][j]*( 2*operators[i][2]*operators[j][2] )/4)
        
        hamiltonians.append(temp)
    
    return hamiltonians
            
    

In [12]:
def get_hamiltonian(number_of_spins,gammas,thetas,distances,operators):
    hamiltonians = get_hamiltonians(number_of_spins,gammas,thetas,distances,operators)
    hamiltonian = 0
    for i in range(number_of_spins):
        for j in range(i+1,number_of_spins):
            hamiltonian+=hamiltonians[i][j]
    return Qobj(hamiltonian)
    

In [ ]:
# thetas1 = [["NA", np.pi/6,np.pi/2],[np.pi/6,"NA",np.pi*5/6],[np.pi/2,np.pi*5/6,"NA"]]
# distances1 = [["NA",1,1],[1,"NA",1],[1,1,"NA"]]
# h = get_hamiltonian(3,[0.1,0.1,0.02],thetas1,distances1,create_operators(3))
# print(h)

In [14]:
def qubit_integrate(rho0,solver, gammas, thetas, distances,number_of_spins):
    
    operators = create_operators(number_of_spins)
    Hdd = get_hamiltonian(number_of_spins, gammas, thetas, distances, operators)
    tlist = np.linspace(0,1000,1000)
    
    c_ops = []

    e_ops = [x for xs in operators for x in xs]
        
    if solver == "me":
        output = mesolve(Hdd, rho0, tlist, c_ops, e_ops)  
    elif solver == "es":
        output = essolve(Hdd, rho0, tlist, c_ops, e_ops)  
    elif solver == "mc":
        ntraj = 250
        output = mcsolve(Hdd, rho0, tlist, c_ops, e_ops,ntraj=ntraj)  
    else:
        raise ValueError("unknown solver")
        
    return output.expect[0], output.expect[1], output.expect[2],output.expect[3], output.expect[4], output.expect[5],output.expect[6], output.expect[7], output.expect[8]

In [15]:
def create_initial_density_matrix(number_of_spins,state_of_spins):
    rho0 = 1
    if state_of_spins[0] == 'x':
        rho0 = 1/2*(qeye(2)+sigmax())
    elif state_of_spins[0] == 'y':
        rho0 = 1/2*(qeye(2)+sigmay())
    elif state_of_spins[0] == 'z':
        rho0 = 1/2*(qeye(2)+sigmaz())
    else:
        rho0 = 1/2*(qeye(2))
    for i in range(1,number_of_spins):
        if state_of_spins[i] == 'x':
            rho0 = tensor(Qobj(rho0),1/2*(qeye(2)+sigmax()))
        elif state_of_spins[i] == 'y':
            rho0 = tensor(Qobj(rho0),1/2*(qeye(2)+sigmay()))
        elif state_of_spins[i] == 'z':
            rho0 = tensor(Qobj(rho0),1/2*(qeye(2)+sigmaz()))
        else:
            rho0 = tensor(Qobj(rho0),1/2*(qeye(2)))
    return Qobj(rho0)


In [16]:
# print(create_initial_density_matrix(3,['x','x','x']))

In [17]:
# print(create_operators(2))

In [18]:
def plot_function(rho0, gammas, thetas, distances, number_of_spins, text):
    # Time list and integration
    tlist = np.linspace(0, 1000, 1000)
    output = qubit_integrate(rho0, "me", gammas, thetas, distances, number_of_spins)

    # Create a grid for subplots (5 rows)
    fig = plt.figure(figsize=(15, 18))

    # Combined graph (row 1)
    ax1 = plt.subplot2grid((5, 1), (0, 0), rowspan=1)
    line_styles = ['-', '--', ':']
    markers = ['o', 'x', 's']

    for spin_idx in range(number_of_spins):
        sx = np.real(output[3 * spin_idx])
        sy = np.real(output[3 * spin_idx + 1])
        sz = np.real(output[3 * spin_idx + 2])

        ax1.plot(
            tlist, sx, label=f"sx_{spin_idx}", linestyle=line_styles[0],
            alpha=0.8, marker=markers[0], markevery=50
        )
        ax1.plot(
            tlist, sy, label=f"sy_{spin_idx}", linestyle=line_styles[1],
            alpha=0.8, marker=markers[1], markevery=50
        )
        ax1.plot(
            tlist, sz, label=f"sz_{spin_idx}", linestyle=line_styles[2],
            alpha=0.8, marker=markers[2], markevery=50
        )

    ax1.set_title('Combined Spin Expectation Values ' + text)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Expectation Values')
    ax1.set_ylim(-1.25, 1.25)
    ax1.legend(loc='upper right', fontsize='small', ncol=2)
    ax1.grid(True, linestyle='--', alpha=0.5)

    # Individual spin graphs (row 2)
    for spin_idx in range(number_of_spins):
        ax = plt.subplot2grid((5, number_of_spins), (1, spin_idx))

        sx = np.real(output[3 * spin_idx])
        sy = np.real(output[3 * spin_idx + 1])
        sz = np.real(output[3 * spin_idx + 2])

        ax.plot(tlist, sx, 'r', label=f"sx_{spin_idx}", linestyle='-', alpha=0.8, marker='o', markevery=50)
        ax.plot(tlist, sy, 'b', label=f"sy_{spin_idx}", linestyle='--', alpha=0.8, marker='x', markevery=50)
        ax.plot(tlist, sz, 'g', label=f"sz_{spin_idx}", linestyle=':', alpha=0.8, marker='s', markevery=50)

        ax.set_title(f'Spin {spin_idx}')
        ax.set_xlabel('Time')
        ax.set_ylabel('Expectation Values')
        ax.set_ylim(-1.25, 1.25)
        ax.legend(loc='upper right', fontsize='x-small')
        ax.grid(True, linestyle='--', alpha=0.5)

    # Fourier transforms (rows 3-5: Sx, Sy, Sz of each spin)
    for row_idx, spin_component in enumerate(['Sx', 'Sy', 'Sz']):
        for spin_idx in range(number_of_spins):
            ax = plt.subplot2grid((5, number_of_spins), (2 + row_idx, spin_idx))

            component = np.real(output[3 * spin_idx + row_idx])  # Sx, Sy, Sz based on row_idx
            freq = np.fft.fftfreq(len(tlist), d=(tlist[1] - tlist[0]))
            window = np.hanning(len(tlist))  # These 2 lines for hamming
            component = component * window # These 2 line for hamming
            component_ft = np.fft.fft(component)
            freq = np.fft.fftshift(freq)
            component_ft = np.fft.fftshift(component_ft)
            ax.plot(freq, np.abs(component_ft), 'r' if spin_component == 'Sx' else ('b' if spin_component == 'Sy' else 'g'))
            ax.set_title(f'{spin_component} Fourier Transform (Spin {spin_idx})')
            ax.set_xlabel('Frequency')
            ax.set_ylabel('Amplitude')
            ax.set_xlim(-0.02, 0.02)
            ax.set_ylim(-0.1,500)
            ax.grid(True, linestyle='--', alpha=0.5)

    # Adjust layout for clarity
    plt.tight_layout()
    filename = f'figs/spin_expectation_values_{text}.png'
    plt.savefig(filename, dpi=300)
    plt.show()


In [19]:
# rhotest = tensor( Qobj(np.array([[1, 0],
#                           [0, 0]])), 
#           qeye(2)/2, qeye(2)/2 ) 
# plot_function(create_initial_density_matrix(3,['x','x','x']),[0.1,0.1,0.1],create_theta(3),distances1,3,"yo")

In [20]:
# for i in range(3):
#     for j in range(3):
#         print(distance_matrix_5[i][j], end=" ")
#     print("\n")

In [21]:

def generate_combinations(letters, n):

    combinations = [''.join(comb) for comb in product(letters, repeat=n)]
    return combinations

# Example usage


# n=3
# combinations = generate_combinations(['x','y', 'z', 'u'], 3)

# # Print the combinations
# # print(f"All combinations of size {n} with letters {letters}:")
# print(combinations)


In [22]:
def check_is_needed(distance_matrix,gammas,state):
    list_1 = ['uuu', 'xuu', 'xyu','xyz','xxy', 'xxz', 'xxu','xxx','yuu','yyx','yyz','yyu','yyy','zuu','zxu','zyu','zzu','zzx','zzy','zzz']
    list_2 = ['uuu', 'xuu', 'uux', 'xyu', 'uxy', 'yux', 'xyz', 'xzy', 'yzx', 'xxy', 'xyx', 'xxz', 'xzx', 'xxu', 'xux', 'xxx', 'yuu', 'uuy', 'yyx', 'yxy', 'yyz', 'yzy', 'yyu', 'yuy', 'yyy', 'zuu', 'uuz', 'zxu', 'zux', 'xuz', 'zyu', 'zuy', 'yuz', 'zzu', 'zuz', 'zzx', 'zxz', 'zzy', 'zyz', 'zzz']
    list_3 = ['uuu', 'xuu', 'uxu', 'xyu', 'yux', 'uxy', 'xyz', 'yxz', 'zxy', 'xxy', 'yxx', 'xxz', 'zxx', 'xxu', 'uxx', 'xxx', 'yuu', 'uyu', 'yyx', 'xyy', 'yyz', 'zyy', 'yyu', 'uyy', 'yyy', 'zuu', 'uzu', 'zxu', 'xzu', 'uzx', 'zyu', 'yzu', 'uzy', 'zzu', 'uzz', 'zzx', 'xzz', 'zzy', 'yzz', 'zzz']


    if distance_matrix[0][1]==1 and distance_matrix[0][2]==1 and distance_matrix[1][2]==1:
        if gammas[1] == 0.1 and gammas[2] == 0.1: #Like Like
            if state in list_1:
                return 1
        elif gammas[1] == 0.1 and gammas[2] == 0.02:
            if state in list_2:
                return 1
        elif gammas[1] == 0.02 and gammas[2] == 0.02:
            if state in list_3:
                return 1
    elif distance_matrix[0][1]==100 and distance_matrix[0][2]==100 and distance_matrix[1][2]==100:
        if gammas[1] == 0.1 and gammas[2] == 0.1:
            if state in list_1:
                return 1
        elif gammas[1] == 0.1 and gammas[2] == 0.02:
            if state in list_2:
                return 1
        elif gammas[1] == 0.02 and gammas[2] == 0.02:
            if state in list_3:
                return 1
    elif distance_matrix[0][1]==1 and distance_matrix[0][2]==100 and distance_matrix[1][2]==100:
        if gammas[1] == 0.1 and gammas[2] == 0.1:
            if state in list_2:
                return 1
        elif gammas[1] == 0.1 and gammas[2] == 0.02:
            if state in list_2:
                return 1
        elif gammas[1] == 0.02 and gammas[2] == 0.02:
            return 1
    elif distance_matrix[0][1]==100 and distance_matrix[0][2]==100 and distance_matrix[1][2]==1:
        if gammas[1] == 0.1 and gammas[2] == 0.1:
            if state in list_3:
                return 1
        elif gammas[1] == 0.1 and gammas[2] == 0.02:
            return 1
        elif gammas[1] == 0.02 and gammas[2] == 0.02:
            if state in list_3:
                return 1
    elif distance_matrix[0][1]==100 and distance_matrix[0][2]==1 and distance_matrix[1][2]==100:
        if gammas[1] == 0.1 and gammas[2] == 0.1:
            return 0
        elif gammas[1] == 0.1 and gammas[2] == 0.02:
            return 1
        elif gammas[1] == 0.02 and gammas[2] == 0.02:
            return 0
    return 0

In [23]:
def computation(number_of_spins):
    letters = ['x','y', 'z', 'u']
    state_of_spins = generate_combinations(letters, number_of_spins)
    
    distance_matrix_1 = [[0,1,1],[1,0,1],[1,1,0]] # [1,1,1]
    distance_matrix_2 = [[0,100,100],[100,0,100],[100,100,0]] # [100,100,100]
    distance_matrix_3 = [[0,1,100],[1,0,100],[100,100,0]] # [1,100,100]
    distance_matrix_4 = [[0,100,100],[100,0,1],[100,1,0]] # [100,100,1]
    distance_matrix_5 = [[0,100,1],[100,0,100],[1,100,0]] # [100,1,100]
    distance_matrices = [distance_matrix_1,distance_matrix_2,distance_matrix_3,distance_matrix_4,distance_matrix_5]
    
    Like_1 = ["like","like"]
    Like_2 = ["like","unlike"]
    Like_3 = ["unlike","unlike"]
    Like_lists = [Like_1,Like_2,Like_3]
    
    thetas = create_theta(number_of_spins)
    
    for distance_matrix in distance_matrices:
        for Like_list in Like_lists:
            for state in state_of_spins:
                gammas = create_gamma(Like_list)
                is_needed = check_is_needed(distance_matrix, gammas, state)
                if is_needed==0:
                    continue
                text = f"Distances {distance_matrix[0][1]} {distance_matrix[0][2]} {distance_matrix[1][2]}; S1 {Like_list[0]}, S2 {Like_list[1]}; State {state[0]} {state[1]} {state[2]}"
                print(text)
                
                rho0 = create_initial_density_matrix(number_of_spins,state)
                plot_function(rho0, gammas, thetas, distance_matrix, number_of_spins,text)
                
                

In [24]:
# computation(3)